<a href="https://colab.research.google.com/github/dataista0/twitch-streams/blob/main/Episodio_5_Colab_RoBERTa_Learning_Rate_Mas_epochs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!git clone https://github.com/dataista0/twitch-streams.git

     |████████████████████████████████| 2.5MB 7.5MB/s 
     |████████████████████████████████| 3.3MB 42.9MB/s 
     |████████████████████████████████| 901kB 31.8MB/s 
Cloning into 'twitch-streams'...
remote: Enumerating objects: 43, done.
remote: Counting objects: 100% (43/43), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 43 (delta 22), reused 28 (delta 10), pack-reused 0
Unpacking objects: 100% (43/43), done.


In [3]:
from google.colab import files
files.upload() # Para subir la api key (kaggle.json)

!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!mkdir -p twitch-streams/data/nlp-getting-started
!kaggle competitions download -c nlp-getting-started -p data/nlp-getting-started/

Saving kaggle.json to kaggle.json
  0% 0.00/22.2k [00:00<?, ?B/s]
100% 22.2k/22.2k [00:00<00:00, 14.8MB/s]
  0% 0.00/965k [00:00<?, ?B/s]
100% 965k/965k [00:00<00:00, 62.2MB/s]
  0% 0.00/411k [00:00<?, ?B/s]
100% 411k/411k [00:00<00:00, 136MB/s]


In [1]:
!ls twitch-streams/

 data
'Episodio 3 - En la nube.ipynb'
'Episodio 3 - Parte 2 - BERT aplicado a Tweets de Desastres.ipynb'
'Episodio 4 - Parte 1 - De 1220 a 832 - Usando todo el dataset de training.ipynb'
'Episodio 4 - Parte 2 - DistilBert 4 Epochs - 0.817, LB 744. Overfitting?.ipynb'
'Episodio 4 - Parte 3 - Checkpoints POC.ipynb'
'Episodio 4 - Parte 4 - Bert varias epochs grabando.ipynb'
 Episodio_5_Agregamos_Colab.ipynb
'Episodio 5 - RoBERTa - 0.82592 - Rank 618.ipynb'
'Episodio 5 - V2 - 3 epochs - RoBERTa  - 0.82807 - 576.ipynb'
 README.md
 requirements_episodio2.txt
'Stream 2 - Razonamiento analogico con Word2Vec y LSH.ipynb'
 stream2-requirements.txt
'Stream 3 - Pipeline de transformers out-of-the-box.ipynb'
'🤗 Transformers Episodio 2 - Under the hood de pipeline.ipynb'
 utils.py


In [2]:
import sys; sys.path.append("twitch-streams/")

In [9]:
get_submission??

In [4]:
import utils
import importlib
importlib.reload(utils)

from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from utils import get_submission, submit, get_tokenizer_and_model,\
                  compute_metrics, load_dfs, tokenize, get_train_args


import transformers
from transformers import Trainer
transformers.logging.set_verbosity_warning()

def read_scores():
    s = !kaggle competitions submissions nlp-getting-started 
    df = pd.DataFrame([(l.split()[0], l.split()[-2]) for l in s[2:]], columns=["Archivo", "Score"]).set_index("Archivo")
    display(df)
    return df


In [5]:
model_id = "episodio_5_colab_2"
pred_file_name = f"{model_id}.csv"
submit_message = "Episodio 5 Roberta Base desde Colab"

model_name = "roberta-base"
tokenizer, model = get_tokenizer_and_model(model_name)

df_base, df_test = load_dfs()
df_train, df_val = train_test_split(df_base, test_size=0.066)
ds_train, ds_val, ds_test = tokenize(tokenizer, df_train, df_val, df_test)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.layer_norm.bias', 'roberta.pooler.dense.bias', 'lm_head.bias', 'roberta.pooler.dense.weight', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

In [6]:
args = get_train_args(model_id, epochs=8, save_steps=1000, evaluation_strategy="steps", eval_steps=200, learning_rate = 0.66*5e-05)

In [7]:
trainer = Trainer(model=model, args=args,
                  train_dataset=ds_train, 
                  eval_dataset=ds_val,
                  compute_metrics=compute_metrics)

trainer.train()

***** Running training *****
  Num examples = 7110
  Num Epochs = 8
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 7112


Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,No log,0.427250,0.833002,0.804651,0.812207,0.797235
400,No log,0.426310,0.834990,0.802850,0.828431,0.778802
600,0.509700,0.463479,0.838966,0.802920,0.850515,0.760369
800,0.509700,0.438011,0.829026,0.777202,0.887574,0.691244
1000,0.470300,0.544764,0.829026,0.798122,0.813397,0.783410
1200,0.470300,0.433074,0.825050,0.769634,0.890909,0.677419
1400,0.470300,0.443881,0.829026,0.796209,0.819512,0.774194
1600,0.439500,0.436682,0.850895,0.812968,0.885870,0.751152
1800,0.439500,0.474910,0.850895,0.818402,0.862245,0.778802
2000,0.382600,0.522682,0.844930,0.812500,0.849246,0.778802


***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
Saving model checkpoint to /content/twitch-streams/data/models/episodio_5_colab_2/checkpoint-1000
Configuration saved in /content/twitch-streams/data/models/episodio_5_colab_2/checkpoint-1000/config.json
Model weights saved in /content/twitch-streams/data/models/episodio_5_colab_2/checkpoint-1000/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
***** Running Evaluation *****
  Num examples = 503
  Batch siz

Step,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
200,No log,0.427250,0.833002,0.804651,0.812207,0.797235
400,No log,0.426310,0.834990,0.802850,0.828431,0.778802
600,0.509700,0.463479,0.838966,0.802920,0.850515,0.760369
800,0.509700,0.438011,0.829026,0.777202,0.887574,0.691244
1000,0.470300,0.544764,0.829026,0.798122,0.813397,0.783410
1200,0.470300,0.433074,0.825050,0.769634,0.890909,0.677419
1400,0.470300,0.443881,0.829026,0.796209,0.819512,0.774194
1600,0.439500,0.436682,0.850895,0.812968,0.885870,0.751152
1800,0.439500,0.474910,0.850895,0.818402,0.862245,0.778802
2000,0.382600,0.522682,0.844930,0.812500,0.849246,0.778802


***** Running Evaluation *****
  Num examples = 503
  Batch size = 8
Saving model checkpoint to /content/twitch-streams/data/models/episodio_5_colab_2/checkpoint-7000
Configuration saved in /content/twitch-streams/data/models/episodio_5_colab_2/checkpoint-7000/config.json
Model weights saved in /content/twitch-streams/data/models/episodio_5_colab_2/checkpoint-7000/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=7112, training_loss=0.2912310794984798, metrics={'train_runtime': 6323.3352, 'train_samples_per_second': 8.995, 'train_steps_per_second': 1.125, 'total_flos': 2.17802681229312e+16, 'train_loss': 0.2912310794984798, 'epoch': 8.0})

In [13]:

#df_res = pd.read_csv('data/nlp-getting-started/sample_submission.csv')
#preds = trainer.predict(ds_test)
import torch.nn.functional as F
import torch
final_preds = F.softmax(torch.from_numpy(preds.predictions), dim=-1)
final_binary_preds = (final_preds[:, 1] > 0.5).numpy().astype(int)
df_res['target'] = final_binary_preds
#df_res.to_csv(PRED_PATH/file_name, index=False)

In [16]:
file_name="pred.csv"
df_res.to_csv(file_name, index=False)

In [19]:
import subprocess
message = "Roberta 8 epochs"
session = subprocess.Popen(['kaggle', 'competitions', 'submit', '-f', file_name, '-m', f'"{message}"', 'nlp-getting-started'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = session.communicate()
print("stdout=")
print(stdout.decode("utf-8"))
print("stderr=")
print(stderr.decode("utf-8"))

stdout=
Successfully submitted to Natural Language Processing with Disaster Tweets
stderr=
100%|##########| 22.2k/22.2k [00:01<00:00, 11.8kB/s]



In [21]:
# Leaderbord - 576 - 0.82807
read_scores();

,Score
Archivo,
-------------------------------------,-----------
pred.csv,0.81857
episodio_5_v2.csv,0.82807
episodio_5_part1-4-epochs-cp2500.csv,0.82592
episodio_5_part1-4-epochs.csv,0.81918
episodio_5_part1.csv,0.82562
episodio_5.csv,0.57033
episodio_4_pred_4.csv,0.81979
episodio_4_pred_2_checkpoint_2000.csv,0.80968


In [22]:
from transformers import AutoModelForSequenceClassification

In [28]:
!ls twitch-streams/data/models/episodio_5_colab_2/checkpoint-4000

checkpoint-1000  checkpoint-3000  checkpoint-5000  checkpoint-7000
checkpoint-2000  checkpoint-4000  checkpoint-6000  runs


In [34]:
p = "twitch-streams/data/models/episodio_5_colab_2/checkpoint-5000"
m = AutoModelForSequenceClassification.from_pretrained(p)

loading configuration file twitch-streams/data/models/episodio_5_colab_2/checkpoint-5000/config.json
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classification",
  "transformers_version": "4.8.1",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file twitch-streams/data/models/episodio_5_colab_2/checkpoint-5000/pytorch_model.bin
All model checkpoint weights were used when initializing Ro

In [35]:
trainer2 = Trainer(model=m, args=args,
                  train_dataset=ds_train, 
                  eval_dataset=ds_val,
                  compute_metrics=compute_metrics)


In [36]:

preds = trainer2.predict(ds_test)
final_preds = F.softmax(torch.from_numpy(preds.predictions), dim=-1)
final_binary_preds = (final_preds[:, 1] > 0.5).numpy().astype(int)
df_res['target'] = final_binary_preds
file_name="pred3.csv"
df_res.to_csv(file_name, index=False)

***** Running Prediction *****
  Num examples = 3263
  Batch size = 8


In [37]:
import subprocess
message = "Roberta 8 epochs - cp 5000"
session = subprocess.Popen(['kaggle', 'competitions', 'submit', '-f', file_name, '-m', f'"{message}"', 'nlp-getting-started'], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
stdout, stderr = session.communicate()
print("stdout=")
print(stdout.decode("utf-8"))
print("stderr=")
print(stderr.decode("utf-8"))

stdout=
Successfully submitted to Natural Language Processing with Disaster Tweets
stderr=
100%|##########| 22.2k/22.2k [00:01<00:00, 16.1kB/s]



In [39]:
read_scores();

,Score
Archivo,
-------------------------------------,-----------
pred2.csv,0.82286
pred2.csv,0.82623
pred.csv,0.81857
episodio_5_v2.csv,0.82807
episodio_5_part1-4-epochs-cp2500.csv,0.82592
episodio_5_part1-4-epochs.csv,0.81918
episodio_5_part1.csv,0.82562
episodio_5.csv,0.57033
